<a href="https://colab.research.google.com/github/taka-onohara/google-colab/blob/main/Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import pandas as pd
import numpy as np

# pwt90 = pd.read_stata('https://www.rug.nl/ggdc/docs/pwt90.dta')
pwt1001 = pd.read_stata('https://dataverse.nl/api/access/datafile/354098')

# data = pwt90.loc[pwt90['country'].isin(["Australia","Austria","Belgium","Canada",
                                        #"Denmark","Finland","France","Germany",
                                        #"Greece","Iceland","Ireland","Italy",
                                        #"Japan","Netherlands","New Zealand","Norway",
                                        #"Portugal","Spain","Sweden","Switzerland",
                                        #"United Kingdom","United States"])][['year', 'countrycode', 'rgdpna', 'rkna', 'emp', 'labsh','avh']]
data = pwt1001.loc[pwt1001['country'].isin(["France","Germany","Canada","Italy","Japan","United Kingdom","United States"])][['year', 'countrycode', 'rgdpna', 'rkna', 'emp', 'labsh']]

# data = data.loc[(data['year'] >= 1960) & (data['year'] <= 2000)].dropna()
data = data.loc[(data['year'] >= 1995) & (data['year'] <= 2019)].dropna()

# Y = rgdpna, K = rkna, L = emp, alpha = 1 - labsh then fix by country, A = (the rest)
data['Y'] = data['rgdpna']
data['K'] = data['rkna']
data['L'] = data['emp']
data['alpha'] = 1 - data['labsh']
data['alpha'] = data.groupby('countrycode')['alpha'].transform('mean')
data['A'] = data['Y'] / ((data['K'] ** data['alpha']) * (data['L'] ** (1 - data['alpha'])))

# confirm Y = test
# data['test'] = data['A'] * (data['K'] ** data['alpha']) * (data['L'] ** (1 - data['alpha']))
# print(data.head(10))

data.drop(data[['rgdpna','rkna','emp','labsh']], axis=1, inplace=True)

# y = Y/L, k = K/L, ln(y) = ln(A) + alpha * ln(k)
data['ln(y)'] = np.log(data['Y'] / data['L'])
data['ln(A)'] = np.log(data['A'])
data['ln(k)'] = np.log(data['K'] / data['L'])

# confirm ln(y) = test1
# data['test1'] = data['ln(A)'] + data['alpha'] * data['ln(k)']
# print(data[['ln(y)','ln(A)','ln(k)','alpha','test1']].head(10))

data = data.sort_values('year')
grouped_data = data.groupby('countrycode')

data['growth_rate'] = grouped_data['ln(y)'].diff() * 100
data['TFP_growth'] = grouped_data['ln(A)'].diff() * 100
data['capital_deepening'] = data['alpha'] * (grouped_data['ln(k)'].diff() * 100)
data['dk/dt'] = grouped_data['ln(k)'].diff() * 100

data = data.dropna()

# confirm growth_rate = test2
# data['test2'] = data['TFP_growth'] + data['capital_deepening']
# print(data[['growth_rate','test2']].head(10))

summary = data.groupby('countrycode').agg({'growth_rate': 'mean',
                                           'TFP_growth':'mean',
                                           'capital_deepening':'mean',
                                           'dk/dt':'mean',
                                           'alpha':'mean'
                                           })
summary['TFP_share'] = summary['TFP_growth'] / summary['growth_rate']
summary['capital_share'] = summary['capital_deepening'] / summary['growth_rate']

# confirm alpha = test3
#summary['test3'] = (summary['growth_rate'] - summary['TFP_growth']) / summary['dk/dt']
#print(summary[['alpha','test3']].head(10))

print('---summary---')
print(summary[['growth_rate','TFP_growth','capital_deepening','TFP_share','capital_share']])

overall = summary[['growth_rate','TFP_growth','capital_deepening']].mean()
overall['TFP_share'] = overall['TFP_growth'] / overall['growth_rate']
overall['capital_share'] = overall['capital_deepening'] / overall['growth_rate']
print('---overall---')
print(overall)

---summary---
             growth_rate  TFP_growth  capital_deepening  TFP_share  \
countrycode                                                          
CAN             0.887700    0.254341           0.633358   0.286517   
DEU             0.661055    0.218340           0.442714   0.330290   
FRA             0.829236    0.260727           0.568510   0.314418   
GBR             1.015544    0.443319           0.572228   0.436534   
ITA            -0.037181   -0.571819           0.534638  15.379181   
JPN             0.710956    0.064154           0.646805   0.090237   
USA             1.536985    0.744728           0.792254   0.484538   

             capital_share  
countrycode                 
CAN               0.713482  
DEU               0.669708  
FRA               0.685583  
GBR               0.563469  
ITA             -14.379178  
JPN               0.909767  
USA               0.515460  
---overall---
growth_rate          0.800614
TFP_growth           0.201970
capital_deepening   